In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from itertools import chain

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Análisis de datos exploratorio
## Caso de estudio: Plant pathology dataset



### Cargar datos


In [ ]:
df_datos = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')
nRow, nCol = df_datos.shape
print(f'There are {nRow} rows and {nCol} columns')
df_datos.sample(10)

### Tipos de datos


In [ ]:
col_labels = 'labels'

print('Tipos de datos: \n', df_datos.dtypes)
print('Tipo de datos de las etiquetas: ', type(df_datos[col_labels][0]))

### Columnas binarias

In [ ]:

## Here you may want to create some extra columns in your table with binary indicators of certain diseases 
## rather than working directly with the 'Finding Labels' column

all_labels = np.unique(list(chain(*df_datos[col_labels].map(lambda x: x.split(' ')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))

for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        df_datos[c_label] = df_datos[col_labels].map(lambda finding: 1.0 if c_label in finding else 0)
df_datos.sample(10)

## Estudio de la población

### Distribución de los hallazgos

In [ ]:
def splitClasses(string):
    return string.split(' ')

def getCountsDataFrame(df, column, labels):
    col = df[column]
    diccionario  = {l:0 for l in labels}
    for element in col:
        lsplit = splitClasses(element)
        #print(lsplit)
        for l_individual in lsplit:
            #print(diccionario[l_individual])
            diccionario[l_individual] += 1
    return diccionario

def plotCounts(counts_dict, graphWidth, name='Counts'):
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    plt.xticks(rotation='vertical')
    plt.title(name)
    plt.bar(*zip(*counts_dict.items()), color=['red', 'green', 'blue', 'cyan', 'black'])
    plt.show()


# Plot disease labels distribution
dict_counts = getCountsDataFrame(df_datos, col_labels, all_labels)

plotCounts(dict_counts, 6, name= 'Distribución de las enfermedades')

#print('Neumonia Cases:', dict_counts['Pneumonia'])
#print('No neumonia Cases:', nRow - dict_counts['Pneumonia'])

In [ ]:
# function to get unique values
def unique(list1):
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def getAllDifferentClasses(dataframe, column):
    labels = dataframe[column].unique()
    anidada = [splitClasses(cs) for cs in labels]
    labels = [l for lista in anidada for l in lista]
    return unique(labels)

def coOcurrencia(dataframe, column):
    #labelsmix = dataframe[column].unique()
    labels = getAllDifferentClasses(dataframe, column)
    n_labels = len(labels)
    matrix_coo = np.zeros((n_labels,n_labels))
    
    for i, la in enumerate(labels):
        max_count = 0
        normalize = False;
        for j, lb in enumerate(labels):
            #search pair (la, lb)
            count = countPairs(dataframe, column, la, lb)
            matrix_coo[i][j] = count
            if count > max_count:
                max_count = count
                normalize = True
        if normalize:
            matrix_coo[i] = matrix_coo[i] * (1/max_count)
    return labels, matrix_coo

def plotCorrelationMatrix(labels, matrix, graphWidth, title='Correlation Matrix for'):
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(matrix, fignum = 1)
    plt.xticks(range(len(labels)), labels, rotation=90)
    plt.yticks(range(len(labels)), labels)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(title, fontsize=15)
    plt.show()
    
def countPairs(df, column, la, lb):
    col = df[column]
    suma = 0
    for element in col:
        lsplit = splitClasses(element)
        if la in lsplit and lb in lsplit:
            suma += 1
    return suma

In [ ]:
labels, matco = coOcurrencia(df_datos, col_labels)
plotCorrelationMatrix(labels, matco, 9, title = 'Matriz de correlación')

## Conclusiones

- Las clases están desbalanceadas
- Etiquetas múltiples (multiclase)
- Solo existe información de la etiqueta